# This solves mazes using a Genetic Algorithm made by Jake Frischmann

In [1]:
import random

class maze:
    maze_array = [
        ["x", "x", "x", "x", "x", "x", "x", "x", "x"],
        ["x", "S", "-", "-", "-", "-", "-", "-", "x"],
        ["x", "-", "x", "-", "x", "-", "x", "-", "x"],
        ["x", "-", "x", "-", "x", "-", "x", "-", "x"],
        ["x", "-", "x", "-", "-", "-", "x", "-", "x"],
        ["x", "-", "x", "-", "x", "x", "x", "-", "x"],
        ["x", "-", "-", "-", "x", "F", "-", "-", "x"],
        ["x", "x", "x", "x", "x", "x", "x", "x", "x"]
    ]
    start_pos = [1, 1]
    finish_pos = [6, 5]
    
class GA:
    move_limit = 10 #10 is best
    total_population_num = 15000 #15000 is best
    mutation_chance = 0.5 #0.5 is best
    population = []
    won = False

class person:
    generation = 1
    moves = []
    fitness_score = 0
    pos = maze.start_pos
    
def print_maze():
    print("Printing maze...")
    for i in range(len(maze.maze_array)):
        print(maze.maze_array[i])
        for w in range(len(maze.maze_array[i])):
            if maze.maze_array[i][w] == "P":
                maze.maze_array[i][w] = "-"
    
        
def winner(player):
    # clean up moves ex. 1, 2, 1, 2, 1, ... simp = 1, ...
    bad_moves_index = []
    for i in range(1, len(player.moves)):
        if player.moves[i] == 1 and player.moves[i - 1] == 2:
            player.moves.pop(i)
            player.moves.pop(i - 1)
            return winner(player)
        if player.moves[i] == 2 and player.moves[i - 1] == 1:
            player.moves.pop(i)
            player.moves.pop(i - 1)
            return winner(player)
        if player.moves[i] == 3 and player.moves[i - 1] == 4:
            player.moves.pop(i)
            player.moves.pop(i - 1)
            return winner(player)
        if player.moves[i] == 4 and player.moves[i - 1] == 3:
            player.moves.pop(i)
            player.moves.pop(i - 1)
            return winner(player)
    print()
    print("These are the winning moves: " + str(player.moves))
    GA.won = True
    return 
            
def move(moving, plr):
    for i in range(len(moving)):
        if moving[i] == 1:
            plr.pos = [plr.pos[0], plr.pos[1] + 1]
        elif moving[i] == 2:
            plr.pos = [plr.pos[0], plr.pos[1] - 1]
        elif moving[i] == 3:
            plr.pos = [plr.pos[0] + 1, plr.pos[1]]
        elif moving[i] == 4:
            plr.pos = [plr.pos[0] - 1, plr.pos[1]]
        if plr.pos[0] >= len(maze.maze_array) or plr.pos[0] < 0 or plr.pos[1] >= len(maze.maze_array[plr.pos[0]]) or plr.pos[1] < 0:
            return 0
        if plr.pos[0] < 0 or plr.pos[1] < 0 or plr.pos[0] > len(maze.maze_array) or  plr.pos[1] > len(maze.maze_array[plr.pos[0]]):
            return 0
        if maze.maze_array[plr.pos[0]][plr.pos[1]] == "F":
            while i != len(plr.moves) - 1:
                plr.moves.pop(len(plr.moves) - 1)
            return "finish"
        if maze.maze_array[plr.pos[0]][plr.pos[1]] == "x":
            return 0
    if maze.maze_array[plr.pos[0]][plr.pos[1]] == "-" or maze.maze_array[plr.pos[0]][plr.pos[1]] == "P":
        return 1
    
def initialize_population(): 
    for i in range(GA.total_population_num):
        GA.population.append(person())
        GA.population[i].moves = []
        for q in range(GA.move_limit):
            GA.population[i].moves.append(random.randint(1, 4))
            
def fitness_evaluation():
    unfit = []
    for i in range(len(GA.population)):
        move_score = move(GA.population[i].moves, GA.population[i])
        if move_score == "finish":
            print
            winner(GA.population[i])
            return 
        elif move_score == 1:
            GA.population[i].fitness_score += 1
        else:
            unfit.append(i)
    fit = []
    for q in range(GA.total_population_num):
        if q not in unfit:
            fit.append(GA.population[q])
            
            
    if len(fit) > 0:
        print("Number of fit people in this generation: " + str(len(fit)))
        print( "An example of fit: " + str(fit[0].moves))
        return "fit"
    else:
        print("No one is fit enough!")
        return "unfit"
        
    GA.population = fit
    
    for i in range(len(GA.population)):
        GA.population[i].fitness_score -= abs(GA.population[i].pos[0] - maze.finish_pos[0]) + abs(GA.population[i].pos[1] - maze.finish_pos[1])
        
def next_generation():
    new_gen = []
    elite = []
    elite.append(GA.population[0])
    for i in range(1, len(GA.population)):
        if GA.population[i].fitness_score > elite[0].fitness_score:
            elite = [GA.population[i]]
        elif GA.population[i].fitness_score == elite[0].fitness_score:
            elite.append(GA.population[i])
    
    for i in range(len(elite)):
        maze.maze_array[elite[i].pos[0]][elite[i].pos[1]] = "P"
        
    num_elite_parents = int(round((GA.total_population_num * GA.mutation_chance) / len(elite), 0) * len(elite))
    num_normal_parents = int(round(0.5 * (GA.total_population_num - num_elite_parents) / len(GA.population), 0) * len(GA.population))
    num_rand_parents = GA.total_population_num - num_normal_parents - num_elite_parents
    
    print("Elite: " + str(num_elite_parents) + "  Normal:"  + str(num_normal_parents) + "  Random: " + str(num_rand_parents))
    
    for i in range(num_elite_parents):
        new_gen.append(person())
        new_gen[i].moves = []
        for w in range(len(elite[i % len(elite)].moves)):
            new_gen[i].moves.append(elite[i % len(elite)].moves[w])
        for q in range(GA.move_limit):
            new_gen[i].moves.append(random.randint(1, 4))
   
    for i in range(num_normal_parents):
        new_gen.append(person())
        new_gen[i + num_elite_parents].moves = []
        for w in range(len(GA.population[(i + num_elite_parents) % len(GA.population)].moves)):
            new_gen[i + num_elite_parents].moves.append(GA.population[(i + num_elite_parents) % len(GA.population)].moves[w])
        for q in range(GA.move_limit):
            new_gen[i + num_elite_parents].moves.append(random.randint(1, 4))
    # generate randomly 
    for i in range(num_rand_parents):
        new_gen.append(person())
        new_gen[i + num_elite_parents + num_normal_parents].moves = []
        for q in range(GA.move_limit * (GA.population[0].generation + 1)):
            new_gen[i + num_elite_parents + num_normal_parents].moves.append(random.randint(1, 4))
            
    
    for i in range(len(new_gen)):
        new_gen[i].generation += 1
        GA.population.append(new_gen[i])
    
    
    
initialize_population()
    
def chek_if_won(gen = 0):
    if GA.won == False:
        print()
        print("Now on to generation " + str(gen))
        next_generation()
        print()
        print_maze()
        print()
        fitness = fitness_evaluation()
        if fitness == "fit":
            gen += 1
            chek_if_won(gen)
        else: 
            initialize_population()
            chek_if_won(gen)
    else:
        print("After " + str(gen) + " generations the solution has been found!")
        
chek_if_won()


Now on to generation 0
Elite: 0  Normal:0  Random: 15000

Printing maze...
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
['x', 'P', '-', '-', '-', '-', '-', '-', 'x']
['x', '-', 'x', '-', 'x', '-', 'x', '-', 'x']
['x', '-', 'x', '-', 'x', '-', 'x', '-', 'x']
['x', '-', 'x', '-', '-', '-', 'x', '-', 'x']
['x', '-', 'x', '-', 'x', 'x', 'x', '-', 'x']
['x', '-', '-', '-', 'x', 'F', '-', '-', 'x']
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']

Number of fit people in this generation: 27
An example of fit: [1, 1, 2, 2, 1, 1, 3, 4, 1, 2]

Now on to generation 1
Elite: 7506  Normal:0  Random: 7494

Printing maze...
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']
['x', 'P', '-', 'P', '-', 'P', '-', '-', 'x']
['x', '-', 'x', '-', 'x', '-', 'x', '-', 'x']
['x', 'P', 'x', 'P', 'x', 'P', 'x', '-', 'x']
['x', '-', 'x', '-', '-', '-', 'x', '-', 'x']
['x', '-', 'x', 'P', 'x', 'x', 'x', '-', 'x']
['x', '-', '-', '-', 'x', 'F', '-', '-', 'x']
['x', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'x']


These are the